In [17]:
from PIL import Image
import torch.nn as nn
import torch
import numpy as np
from torchvision import transforms
from torch.autograd import Variable
import torch.nn.functional as F

img = Image.open('FH3_1_cvsr.png')



In [6]:
h = img.size[1]
w = img.size[0]

In [7]:
T=transforms.Compose([transforms.ToTensor()])
P=transforms.Compose([transforms.ToPILImage()])

ten=torch.unbind(T(img))
x=ten[0].unsqueeze(0).unsqueeze(0)

a=np.array([[1, 0, -1],[2,0,-2],[1,0,-1]])
conv1=nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1, bias=False)
conv1.weight=nn.Parameter(torch.from_numpy(a).float().unsqueeze(0).unsqueeze(0))

G_x=conv1(Variable(x)).data.view(1,h,w)

b=np.array([[1, 2, 1],[0,0,0],[-1,-2,-1]])
conv2=nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1, bias=False)
conv2.weight=nn.Parameter(torch.from_numpy(b).float().unsqueeze(0).unsqueeze(0))
G_y=conv2(Variable(x)).data.view(1,h,w)

G=torch.sqrt(torch.pow(G_x,2)+ torch.pow(G_y,2))
X=P(G)
X.save('grad.png')

In [12]:
x.shape

torch.Size([1, 1, 1080, 1920])

In [33]:
class Get_gradient(nn.Module):
    def __init__(self):
        super(Get_gradient, self).__init__()
        kernel_v = [[0, -1, 0], 
                    [0, 0, 0], 
                    [0, 1, 0]]
        kernel_h = [[0, 0, 0], 
                    [-1, 0, 1], 
                    [0, 0, 0]]
        kernel_h = torch.FloatTensor(kernel_h).unsqueeze(0).unsqueeze(0)
        kernel_v = torch.FloatTensor(kernel_v).unsqueeze(0).unsqueeze(0)
        self.weight_h = nn.Parameter(data = kernel_h, requires_grad = False)
        self.weight_v = nn.Parameter(data = kernel_v, requires_grad = False)

    def forward(self, x):
        x0 = x[:, 0, :]
        x1 = x[:, 1, :]
        x2 = x[:, 2, :]
        x0_v = F.conv2d(x0.unsqueeze(1), self.weight_v, padding=1)
        x0_h = F.conv2d(x0.unsqueeze(1), self.weight_h, padding=1)

        x1_v = F.conv2d(x1.unsqueeze(1), self.weight_v, padding=1)
        x1_h = F.conv2d(x1.unsqueeze(1), self.weight_h, padding=1)

        x2_v = F.conv2d(x2.unsqueeze(1), self.weight_v, padding=1)
        x2_h = F.conv2d(x2.unsqueeze(1), self.weight_h, padding=1)

        x0 = torch.sqrt(torch.pow(x0_v, 2) + torch.pow(x0_h, 2) + 1e-6)
        x1 = torch.sqrt(torch.pow(x1_v, 2) + torch.pow(x1_h, 2) + 1e-6)
        x2 = torch.sqrt(torch.pow(x2_v, 2) + torch.pow(x2_h, 2) + 1e-6)

        x = torch.cat([x0, x1, x2], dim=1)
        return x

In [34]:
gradfunc = Get_gradient()

In [35]:
data = T(img)
data = data.unsqueeze(0)
data.shape

torch.Size([1, 3, 1080, 1920])

In [36]:
gx = gradfunc(data)

In [37]:
gx.shape

torch.Size([1, 3, 1080, 1920])

In [38]:
out_img = P(gx.squeeze(0))
out_img.save('gx.png')

In [39]:
data_aug = data + gx

In [40]:
img_aug = P(data_aug.squeeze(0))
img_aug.save('aug.png')

In [50]:
import cv2
import numpy as np
import math

moon = cv2.imread("FH3_1_cvsr.png", 0)

moon.shape

(1080, 1920)

In [51]:
row, column = moon.shape
moon_f = np.copy(moon)
moon_f = moon_f.astype("float")

gradient = np.zeros((row, column))

# for x in range(row - 1):
#     for y in range(column - 1):
#         gx = abs(moon_f[x + 1, y] - moon_f[x, y])
#         gy = abs(moon_f[x, y + 1] - moon_f[x, y])
#         gradient[x, y] = gx + gy
for x in range(1, row - 1):
    for y in range(1, column - 1):
        gx = abs(moon_f[x + 1, y] - moon_f[x - 1, y])
        gy = abs(moon_f[x, y + 1] - moon_f[x, y - 1])
        gradient[x, y] = math.sqrt(gx**2 + gy**2)

sharp = moon_f + gradient
sharp = np.where(sharp < 0, 0, np.where(sharp > 255, 255, sharp))

gradient = gradient.astype("uint8")
sharp = sharp.astype("uint8")
cv2.imshow("moon", moon)
cv2.imshow("gradient", gradient)
cv2.imshow("sharp", sharp)
cv2.waitKey()

-1